Total Shipment Weight

Last Updated: July 2025
Easy
ID 2058
76

Amazon

Calculate the total weight for each shipment and add it as a new column. Your output needs to have all the existing rows and columns in addition to the  new column that shows the total weight for each shipment. One shipment can have multiple rows.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

spark = SparkSession.builder.appName("Shipment").getOrCreate()

schema = StructType( [   StructField("shipmet",IntegerType(), True) ,
                       StructField("sub_id", IntegerType(), True), 
                       StructField("weight", IntegerType(), True), 
                        StructField("shipment_date", StringType(), True)  ])

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/17 01:07:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = [
    (101, 1, 10, "2021-08-30"),
    (101, 2, 20, "2021-09-01"),
    (101, 3, 10, "2021-09-05"),
    (102, 1, 50, "2021-09-02"),
    (103, 1, 25, "2021-09-01"),
    (103, 2, 30, "2021-09-02"),
    (104, 1, 30, "2021-08-25"),
    (104, 2, 10, "2021-08-26"),
    (105, 1, 20, "2021-09-02")
]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

In [3]:
df.show()

+-------+------+------+-------------+
|shipmet|sub_id|weight|shipment_date|
+-------+------+------+-------------+
|    101|     1|    10|   2021-08-30|
|    101|     2|    20|   2021-09-01|
|    101|     3|    10|   2021-09-05|
|    102|     1|    50|   2021-09-02|
|    103|     1|    25|   2021-09-01|
|    103|     2|    30|   2021-09-02|
|    104|     1|    30|   2021-08-25|
|    104|     2|    10|   2021-08-26|
|    105|     1|    20|   2021-09-02|
+-------+------+------+-------------+



In [4]:
df.printSchema()

root
 |-- shipmet: integer (nullable = true)
 |-- sub_id: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- shipment_date: string (nullable = true)



# 1

In [15]:
from pyspark.sql import functions as F 

# group by the shipment column name in your df
df_total = df.groupBy("shipmet") \
             .agg(F.sum("weight").alias("total_weight"))

# join back to original df
df1 = df.join(df_total, on="shipmet", how="left") \
        .orderBy("shipmet", "sub_id", "weight", "shipment_date")

df1.show()


+-------+------+------+-------------+------------+
|shipmet|sub_id|weight|shipment_date|total_weight|
+-------+------+------+-------------+------------+
|    101|     1|    10|   2021-08-30|          40|
|    101|     2|    20|   2021-09-01|          40|
|    101|     3|    10|   2021-09-05|          40|
|    102|     1|    50|   2021-09-02|          50|
|    103|     1|    25|   2021-09-01|          55|
|    103|     2|    30|   2021-09-02|          55|
|    104|     1|    30|   2021-08-25|          40|
|    104|     2|    10|   2021-08-26|          40|
|    105|     1|    20|   2021-09-02|          20|
+-------+------+------+-------------+------------+



# 2

In [5]:

from pyspark.sql import functions as F 
from pyspark.sql.window import Window 

# Define window partitioned by shipment_id
window_spec = Window.partitionBy(df.shipmet)

# Add total weight  per shipment 

df1 = df.withColumn("total_weitght", F.sum(df.weight).over(window_spec) ).orderBy(df.shipmet,df.sub_id,df.weight,df.shipment_date)
df1.show()

+-------+------+------+-------------+-------------+
|shipmet|sub_id|weight|shipment_date|total_weitght|
+-------+------+------+-------------+-------------+
|    101|     1|    10|   2021-08-30|           40|
|    101|     2|    20|   2021-09-01|           40|
|    101|     3|    10|   2021-09-05|           40|
|    102|     1|    50|   2021-09-02|           50|
|    103|     1|    25|   2021-09-01|           55|
|    103|     2|    30|   2021-09-02|           55|
|    104|     1|    30|   2021-08-25|           40|
|    104|     2|    10|   2021-08-26|           40|
|    105|     1|    20|   2021-09-02|           20|
+-------+------+------+-------------+-------------+

